## Рассчет стоимости 


Будем использовать готовые и открытые API:
1. OCR - yandex cloud -> 0.12 рублей за фоток
2. LLM - OpenRouter -> цена зависит от модели

In [1]:
text_receipt = """
ООО "ПРОДТОРГ"

690033, г. Владивосток, пр-кт 100-летия Владивостока, д. 68

КАССОВЫЙ ЧЕК 142

(ПРИХОД)

145.77

=145.77

9

9.99

*1

ХАРБИН СВ

139.90

139.90

1

ПАКЕТ РЕМИ сити 380

[M+10491 ВОДА ДЕТСКАЯ Н ДЕТО 19

[M+18076 ПИВО ХАРБИН СВ

[M+17732 ПИВО

M+12661 KBAC ЖИВОЙ АРС 149.98

M+16126

16126 KBAC ЖИВОГО БР 148.79

19797 НАПИТОК ЭНЕРГЕ 119.90

=139.90

=139.90

=149.98

=148.79

=119.90

*1.0

1

1

=41.76

=86.98

=55.90

=56.90

23.63

55.90

56.90

69.90 0.338

259.90 *0.142

229.90 *0.258

77.98

1

001095шт

153591шт

017921UT

017921шт

003779шт

148431шт

198822шт

045006шт

121403шт

181207шт

181206шт

032784кг

066284кг

066061кг

194296шт

081834шт

167506шт

004031кг

САЛФЕТКИ ВЛАЖНЫЕ ONE T 86.98

САЛФЕТКИ БУМАЖНЫЕ ДИВН 41.76

ЛЕПЕШКА С ЧЕСНОКОМ 350

ПЕПЕШКА УЗБЕКСКАЯ 350Г

ОГУРЦЫ ГРУНТОВЫЕ ПРИМОР

ПЕРЕЦ СЛАДКИЙ ОРАНЖЕВЫЙ

ПОМИДОРЫ ЧЕРРИ РОЗОВЫЕ

МОРОЖЕНОЕ МЕЛОНА ПУРПУ

[M+18969 МОРОЖЕНОЕ МELO 90.99

МОРОЖЕНОЕ БИНГРЭ БОДРЯ 94.98

ПЕРЕЦ СЛАДКИЙ КРАСНЫЙ К 189.90 0.346

=36.91

=59.31

=77.98

=90.99

=94.98

=65.71

БАНКОВСКИЕ ОПЛАТЫ

000 Продторг

г. Владивосток, Приморский край

пр-кт 100-летия Владивостока

д. 68

11:53

ЧЕК

0105

Оплата

Мерчант: 701000111801

(E4) *5977

A0000006581010

1545.28

06.07.25

ПАО СБЕРБАНК

Терминал: 32662488

MIR

Сумма (Руб):

Комиссия за операцию 0 руб.

ОДОБРЕНО

K/A: 059518

518755305974

Проверено на устройстве клиента

B51807A736F7DFCEB9299FC8C9070488B367819B

Итог

БЕЗНАЛИЧНЫМИ

СУММА НДС 20%

=1545.28

=1545.28

=155.38 СУММА НДС 10%

=55.73

=118.24

ВЫ СЭКОНОМИЛИ, руб

Спасибо за покупку!

Количество наклеек 2 шт.

Ваша карта лояльности 2555000676659

Активный баланс 185.0 бонусов

Вам начислено 46.4 бонусов

У Вас списано 0.0 бонусов

КАССИР Хайдарова

КАССИИ Хандарова 1og.gov.ru

САЙТ ФНС www.nalog govеми Сити"

МЕСТО РАСЧЕТОВ Ма1085 06.07.25 11:53 KACCA 0002.01 СМЕНА 1085 06

CHO OCH

ОД 128408 оп 0860959236

3H KKT 0128226843 он 728 2543111571
"""

In [2]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)



In [3]:
ocr_ya = 0.12   # Стоимость OCR для одного чека
count_receipts = 1 # Количество чеков для обработки
dollar_to_ruble = 80  # Примерный курс доллара к рублю

In [4]:
prompt_text = f"""
# Role
Ты — ассистент по извлечению данных из текстов чеков о покупках. Тебе дан текст чека, который может содержать список покупок, их стоимость и другую информацию (даты, адреса, скидки, итоговые суммы и т.д.).

# Goal
Твоя задача — выделить из чека только следующие данные для каждой купленной позиции:
-позиция — наименование товара или услуги
-количество — количество единиц товара или услуги (если указано)
=цена позиции — итоговая цена этого товара или услуги (без учета скидок, если указано отдельно)

# Instruction
1. Игнорируй итоги, суммы по скидкам, налоги, способы оплаты и прочую информацию, не являющуюся отдельной покупкой.
2. Если у позиции указано количество и цена за штуку, указывай итоговую цену (цена × количество).
3. Не включай товары с нулевой ценой или подарки.
4. Сохраняй структуру: одна строка — одна позиция.

# Формат ответа:
Для каждого чека составь список в формате:
Название позиции - количество - цена итоговая

# Чеки
# Чек 1
{text_receipt*count_receipts}
"""

In [5]:
output_text_receipt = """ХАРБИН СВ - 139.90 - K x Цена за позицию
ПАКЕТ РЕМИ СИТИ 380 - 9.99
ВОДА ДЕТСКАЯ Н ДЕТО 19 - 10491
ПИВО ХАРБИН СВ - 18076
ПИВО - 17732
КВАС ЖИВОЙ АРС - 149.98
КВАС ЖИВОЙ БР - 148.79
НАПИТОК ЭНЕРГЕ - 119.90
САЛФЕТКИ ВЛАЖНЫЕ ONE T - 86.98
САЛФЕТКИ БУМАЖНЫЕ ДИВН - 41.76
ЛЕПЕШКА С ЧЕСНОКОМ 350 - 55.90
ЛЕПЕШКА УЗБЕКСКАЯ 350Г - 56.90
ОГУРЦЫ ГРУНТОВЫЕ ПРИМОР - 69.90
ПЕРЕЦ СЛАДКИЙ ОРАНЖЕВЫЙ - 259.90
ПОМИДОРЫ ЧЕРРИ РОЗОВЫЕ - 229.90
МОРОЖЕНОЕ МЕЛОНА ПУРПУ - 77.98
МОРОЖЕНОЕ MELO - 90.99
МОРОЖЕНОЕ БИНГРЭ БОДРЯ - 94.98
ПЕРЕЦ СЛАДКИЙ КРАСНЫЙ К - 65.71""" * count_receipts

In [6]:
prompt_input_tokens = count_tokens(prompt_text)
prompt_output_tokens = count_tokens(prompt_text)

input_cost_r1 = 0.50
output_cost_r1 = 2.15

input_cost_r1 = 0.13
output_cost_r1 = 0.60


prompt_input_tokens_cost = input_cost_r1 * dollar_to_ruble / 10**6 # стоимость запроса к LLM в долларах за 1 млн токенов
prompt_output_tokens_cost = output_cost_r1 * dollar_to_ruble / 10**6 # стоимость запроса к LLM в долларах за 1 млн токенов


prompt_input_tokens_cost_rub = prompt_input_tokens * prompt_input_tokens_cost
prompt_output_tokens_cost_rub = prompt_output_tokens * prompt_output_tokens_cost

ocr_total_rub = ocr_ya * count_receipts
total_rub = ocr_total_rub + prompt_input_tokens_cost_rub + prompt_output_tokens_cost_rub

In [7]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели DeepSeek R1:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 1 чеков модели DeepSeek R1:

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.01 ₽
  LLM обработка (output):        0.06 ₽
  --------------------------------------
  Итого:                         0.19 ₽


In [8]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели Qwen3 MOE:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 1 чеков модели Qwen3 MOE:

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.01 ₽
  LLM обработка (output):        0.06 ₽
  --------------------------------------
  Итого:                         0.19 ₽


In [9]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели Qwen3 MOE:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 1 чеков модели Qwen3 MOE:

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.01 ₽
  LLM обработка (output):        0.06 ₽
  --------------------------------------
  Итого:                         0.19 ₽


In [10]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели Qwen3 MOE:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 1 чеков модели Qwen3 MOE:

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.01 ₽
  LLM обработка (output):        0.06 ₽
  --------------------------------------
  Итого:                         0.19 ₽


In [22]:
from math import ceil

def calculate_image_tokens(width: int, height: int, model: str = "gpt-4o"):
    """
    width, height — оригинальные размеры изображения
    model — "gpt-4o" или "gpt-4o-mini"
    Возвращает: (tokens, cost_usd)
    """
    # Шаг 1: масштабирование для вписывания в рамку 2048×2048
    w, h = width, height
    if w > 2048 or h > 2048:
        aspect = w / h
        if aspect > 1:
            w, h = 2048, int(2048 / aspect)
        else:
            w, h = int(2048 * aspect), 2048

    # Шаг 2: масштабирование, чтобы меньшая сторона была не меньше 768px
    if w >= h and h > 768:
        w, h = int((768 / h) * w), 768
    elif h > w and w > 768:
        w, h = 768, int((768 / w) * h)

    tiles_w = ceil(w / 512)
    tiles_h = ceil(h / 512)
    n_tiles = tiles_w * tiles_h

    if model.lower() in ("gpt-4o", "gpt-4o turbo", "gpt-4"):
        base = 85
        cost_per_tile = 170
        cost_per_token_usd = 2.5 / 1_000_000  # ~ $2.50 за 1M input‑токенов
    elif model.lower() in ("gpt-4o-mini",):
        base = 2833
        cost_per_tile = 5667
        cost_per_token_usd = 0.15 / 1_000_000  # ~ $0.15 за 1M токенов
    else:
        raise ValueError("Unknown model")

    token_count = base + cost_per_tile * n_tiles
    cost = token_count * cost_per_token_usd
    return token_count, cost

# Пример:
tokens, usd = calculate_image_tokens(1024, 1536, model="gpt-4o")
print(tokens, usd)  # tokens=1105, cost≈0.00000276 $ (~0.0000028 USD)


1105 0.0027625


In [ ]:
def calculate_receipt_processing_cost(
    text_receipt: str,
    count_receipts: int = 1,
    ocr_cost_per_receipt: float = 0.12,
    dollar_to_ruble: float = 80,
    input_cost_r1: float = 0.13,
    output_cost_r1: float = 0.60,
    name_model: str = "DeepSeek R1",
    count_tokens_func=None
    
):
    if count_tokens_func is None:
        raise ValueError("Необходимо передать функцию подсчёта токенов через аргумент `count_tokens_func`")

    prompt_text = f"""
# Role
Ты — ассистент по извлечению данных из текстов чеков о покупках. Тебе дан текст чека, который может содержать список покупок, их стоимость и другую информацию (даты, адреса, скидки, итоговые суммы и т.д.).

# Goal
Твоя задача — выделить из чека только следующие данные для каждой купленной позиции:
-позиция — наименование товара или услуги
=цена позиции — итоговая цена этого товара или услуги (без учета скидок, если указано отдельно)

#Instruction
1. Игнорируй итоги, суммы по скидкам, налоги, способы оплаты и прочую информацию, не являющуюся отдельной покупкой.
2. Если у позиции указано количество и цена за штуку, указывай итоговую цену (цена × количество).
3. Не включай товары с нулевой ценой или подарки.
4. Сохраняй структуру: одна строка — одна позиция.

# Формат ответа:
Для каждого чека составь список в формате:
Название позиции - цена итоговая

# Чеки
# Чек 1
{text_receipt * count_receipts}
"""

    # Подсчёт токенов
    input_tokens = 112285 * count_receipts
    print(f"Input tokens: {input_tokens}")
    #input_tokens = count_tokens_func(prompt_text)
    #output_tokens = count_tokens(text_receipt)  # Примерно предполагается равный размер
    output_tokens = 443  # Примерно предполагается равный размер

    # Стоимость токенов (в рублях)
    input_cost_per_token_rub = input_cost_r1 * dollar_to_ruble / 1_000_000
    output_cost_per_token_rub = output_cost_r1 * dollar_to_ruble / 1_000_000

    input_cost_rub = input_tokens * input_cost_per_token_rub
    output_cost_rub = output_tokens * output_cost_per_token_rub

    ocr_total_rub = ocr_cost_per_receipt * count_receipts
    total_rub = ocr_total_rub + input_cost_rub + output_cost_rub

    # Красивый вывод
    print(f"Расчёт стоимости для {count_receipts} чеков модели {name_model}\n")
    print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
    print(f"  LLM обработка (input):     {input_cost_rub:8.2f} ₽")
    print(f"  LLM обработка (output):    {output_cost_rub:8.2f} ₽")
    print("  --------------------------------------")
    print(f"  Итого:                     {total_rub:8.2f} ₽")

    # return {
    #     "ocr_rub": ocr_total_rub,
    #     "llm_input_rub": input_cost_rub,
    #     "llm_output_rub": output_cost_rub,
    #     "total_rub": total_rub,
    #     "input_tokens": input_tokens,
    #     "output_tokens": output_tokens
    # }


In [12]:
# 112285
# 204085

In [13]:
import tiktoken

def count_tokens(text: str, encoding_name: str = "o200k_base"):
    enc = tiktoken.get_encoding(encoding_name)
    return len(enc.encode(text))


In [14]:
text_receipt = """
Творог 5% "Пискаре" - 45.40
Творог 5% "Пискаре" - 45.40
Пельмени Снежная - 206.95
Ананасы кусочки - 48.90
Молоко отбор, пасте - 39.40
Молоко отбор, пасте - 39.40
Компот Bui Foods Ви - 104.00
Сыр Брынза Болгарс - 66.46
Голень шипленка ох - 183.62
Соте из баклажан - 113.02
Мандарин Испания - 166.32
Йогурт Валио 0,12 - 23.90
Йогурт Валио фрукт - 23.90
Йогурт Валио фрукт - 22.90
Йогурт Валио фрукт - 22.90
Йогурт н/лакт Вали - 22.90
Йогурт Валио - 22.90
Йогурт питьевой Дл - 23.90
Яблоки сезонные - 71.22
Бананы - 53.54
Зубная паста Лакал - 119.40
Зубная паста Лакал - 125.40
Набор игрушек бси. - 9.90
Чебуреки Классичес - 69.60
Украшение "Бусы" 6 - 9.90
Украшение "Бусы" 6 - 9.90
Томаты тепличные - 62.29
Огурцы тепличные - 51.25
Сметана ВАЛИО дона - 29.90
DOVE Крен-тимло Рос - 37.40
Салат "Мивая сила" - 33.43
Сосиски "Сливочные" - 92.90
"""

In [15]:
count_tokens(text_receipt)

443

In [16]:
for k in range(1, 11):
    calculate_receipt_processing_cost(
        text_receipt=text_receipt,
        count_receipts=k,
        ocr_cost_per_receipt=ocr_ya,
        dollar_to_ruble=dollar_to_ruble,
        input_cost_r1=0.5,
        output_cost_r1=2.15,
        name_model="DeepSeek R1",
        count_tokens_func=count_tokens
    )


Input tokens: 112285
Расчёт стоимости для 1 чеков модели DeepSeek R1

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         4.49 ₽
  LLM обработка (output):        0.08 ₽
  --------------------------------------
  Итого:                         4.69 ₽
Input tokens: 224570
Расчёт стоимости для 2 чеков модели DeepSeek R1

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         8.98 ₽
  LLM обработка (output):        0.08 ₽
  --------------------------------------
  Итого:                         9.30 ₽
Input tokens: 336855
Расчёт стоимости для 3 чеков модели DeepSeek R1

  OCR обработка чеков:           0.36 ₽
  LLM обработка (input):        13.47 ₽
  LLM обработка (output):        0.08 ₽
  --------------------------------------
  Итого:                        13.91 ₽
Input tokens: 449140
Расчёт стоимости для 4 чеков модели DeepSeek R1

  OCR обработка чеков:           0.48 ₽
  LLM обработка (input):        17.97 ₽
  LLM обработка (output):        0.08

In [17]:
for k in range(1, 11):
    calculate_receipt_processing_cost(
        text_receipt=text_receipt,
        count_receipts=k,
        ocr_cost_per_receipt=ocr_ya,
        dollar_to_ruble=dollar_to_ruble,
        input_cost_r1=0.50,
        output_cost_r1=2.15,
        name_model="DeepSeek R1",
        count_tokens_func=count_tokens
    )


Input tokens: 112285
Расчёт стоимости для 1 чеков модели DeepSeek R1

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         4.49 ₽
  LLM обработка (output):        0.08 ₽
  --------------------------------------
  Итого:                         4.69 ₽
Input tokens: 224570
Расчёт стоимости для 2 чеков модели DeepSeek R1

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         8.98 ₽
  LLM обработка (output):        0.08 ₽
  --------------------------------------
  Итого:                         9.30 ₽
Input tokens: 336855
Расчёт стоимости для 3 чеков модели DeepSeek R1

  OCR обработка чеков:           0.36 ₽
  LLM обработка (input):        13.47 ₽
  LLM обработка (output):        0.08 ₽
  --------------------------------------
  Итого:                        13.91 ₽
Input tokens: 449140
Расчёт стоимости для 4 чеков модели DeepSeek R1

  OCR обработка чеков:           0.48 ₽
  LLM обработка (input):        17.97 ₽
  LLM обработка (output):        0.08

In [18]:
for k in range(1, 11):
    calculate_receipt_processing_cost(
        text_receipt=text_receipt,
        count_receipts=k,
        ocr_cost_per_receipt=ocr_ya,
        dollar_to_ruble=dollar_to_ruble,
        input_cost_r1=0.13,
        output_cost_r1=0.60,
        name_model="Qwen 3 MOE",
        count_tokens_func=count_tokens
    )


Input tokens: 112285
Расчёт стоимости для 1 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         1.17 ₽
  LLM обработка (output):        0.02 ₽
  --------------------------------------
  Итого:                         1.31 ₽
Input tokens: 224570
Расчёт стоимости для 2 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         2.34 ₽
  LLM обработка (output):        0.02 ₽
  --------------------------------------
  Итого:                         2.60 ₽
Input tokens: 336855
Расчёт стоимости для 3 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.36 ₽
  LLM обработка (input):         3.50 ₽
  LLM обработка (output):        0.02 ₽
  --------------------------------------
  Итого:                         3.88 ₽
Input tokens: 449140
Расчёт стоимости для 4 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.48 ₽
  LLM обработка (input):         4.67 ₽
  LLM обработка (output):        0.02 ₽
 

In [19]:
from PIL import Image

def inspect_image(path):
    with Image.open(path) as img:
        fmt = img.format       # например, 'JPEG', 'PNG'
        width, height = img.size  # размеры в пикселях
        info = img.info   # словарь с метаданными (например, dpi)
        dpi = info.get('dpi', None)
        # Также можно использовать img.info.get('resolution') или img.tag для TIFF
    return fmt, width, height, dpi

path = 'dataset/receipt_good.jpg'
fmt, w, h, dpi = inspect_image(path)
print(f"Формат: {fmt}")
print(f"Размер: {w}×{h} px")
if dpi:
    print(f"Плотность: {dpi[0]}×{dpi[1]} dpi")
else:
    print("Информация о DPI отсутствует")


Формат: JPEG
Размер: 700×933 px
Информация о DPI отсутствует


In [20]:
import math
from openai import OpenAI  # или import openai

def estimate_image_tokens(width: int, height: int) -> int:
    tiles_x = math.ceil(width / 32)
    tiles_y = math.ceil(height / 32)
    n = tiles_x * tiles_y
    return 85 + 170 * n

width = 700
height = 933
tokens = estimate_image_tokens(width, height)
print(f"Tiles X: {math.ceil(width/32)}, Tiles Y: {math.ceil(height/32)}")
print(f"Предполагаемое число токенов: {tokens}")


Tiles X: 22, Tiles Y: 30
Предполагаемое число токенов: 112285


In [21]:
from PIL import Image

img = Image.open('dataset/receipt_bad.jpg')  # или .png, .webp и т.д.

# Новый размер (например, уменьшить до 256×256)
new_width = 960 // 2
new_height = 1280 // 2

# ЛУЧШЕЕ КАЧЕСТВО при уменьшении — resample=Image.LANCZOS
img_resized = img.resize((new_width, new_height), resample=Image.LANCZOS)
img_resized.save('dataset/receipt_bad_new.jpg')
